In [3]:
#imports
import json
import requests
import pandas as pd
from bs4 import BeautifulSoup
import regex as re

In [16]:
def gettable(year):
    url = 'http://laschoolboard.org/event/select?date_filter%5Bvalue%5D%5Byear%5D={}&date_filter%5Bvalue%5D%5Bmonth%5D='.format(year) 
    r = requests.get(url)
    #looking for the container that has the table with all the information 
    soup = BeautifulSoup(r.content)
    a = soup.find('body').find('div', id = 'middlecontainer').find('div', id = 'squeeze-content').find('div').find('table', class_ = 'views-table cols-3').find('tbody')


    #3 columns of information
    mtgdates = []
    mtgnames = []
    labels = []
    files = []

    #loop through each row in the table
    for x in a.find_all('tr'):
        #mtgdate and name all have consistent class names
        mtgdate = x.find('td', class_ = 'views-field views-field-unix-event-start').get_text().strip()
        mtgname = x.find('td', class_ = 'views-field views-field-title').get_text().strip()
    
        #find links and loop through, add to list of links
        m = x.find('td', class_ = 'views-field views-field-all-files')
        names = []
        links = []
        findlinks = m.find_all('a')
        for link in findlinks:
            #get name of pdf
            names.append(link.text)
            #get href tag to get directly to the link of the pdf
            #can use these links later to call the actual data
            links.append(link.get('href'))
        #add row to the list
        mtgdates.append(mtgdate)
        mtgnames.append(mtgname)
        labels.append(names)
        files.append(links)
    #make dataframe out of all the rows - one dataframe represents one year
    return pd.DataFrame({'mtgdates': mtgdates, 'mtgnames': mtgnames,'names':labels, 'files': files})

In [118]:
#code from https://stackoverflow.com/questions/22800100/parsing-a-pdf-via-url-with-python-using-pdfminer
import urllib
import regex as re
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
from io import StringIO
from io import BytesIO

def pdf_from_url_to_txt(url):
    rsrcmgr = PDFResourceManager()
    retstr = StringIO()
    codec = 'utf-8'
    laparams = LAParams()
    device = TextConverter(rsrcmgr, retstr, codec=codec, laparams=laparams)
    f = urllib.request.urlopen(url).read()
    fp = BytesIO(f)
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    password = ""
    maxpages = 0
    caching = True
    pagenos = set()
    for page in PDFPage.get_pages(fp,
                                  pagenos,
                                  maxpages=maxpages,
                                  password=password,
                                  caching=caching,
                                  check_extractable=True):
        interpreter.process_page(page)
    fp.close()
    device.close()
    str = retstr.getvalue()
    retstr.close()
    return str

def application (pdfurls):
    texts = []
    if len(pdfurls) > 0:
        for pdfurl in pdfurls:
            text = pdf_from_url_to_txt(re.sub(r" ", "%20", pdfurl))
            text_wordsonly = re.sub(r"[^A-z\s]", "", text)
            text_wordsonly = re.sub(r"\s+", " ", text_wordsonly)
            texts.append(text_wordsonly)
            #adds all the text to file - problem- if rerun does not overwrite file - need to fix this
            with open("{}.txt".format(year), "a+") as text_file:
                text_file.write(text_wordsonly + "\n")
    return texts
#http://laschoolboard.org/sites/default/files/127%20(Charter)%20Approval%20of%20Animo%20Charter%20MS1.pdf

In [120]:
year = 2017
df = gettable(year)
df['text'] = df.files.apply(application)
df

KeyboardInterrupt: 

In [116]:
### this should work but doesn't due to errors in 2011 and 2016 - once fixed should work
dfs = []
for year in range(2010,2022):
    df_ = gettable(year)
    df_['files'] = df_.files.apply(application)
    dfs.append(year:{df_})

### UPDATES

Still some errors but getting better

> btw some files are unreadable - but I dont think thats fixable, they seem scanned in 

> need to catch and handle exception that happens in 2011 and 2016 

>perhaps create iteration from 2010-2022 that does this automatically - as currently only saves in textfiles and not in dataframes. 

>Maybe can add all to one dataframe- dates are already in it so would be fine --

>some of the files are really long and take a long time (1000 + pages)

In [14]:
#df.to_pickle('links12.pandas')